In [5]:
import requests
from bs4 import BeautifulSoup
import random
import time

# 크롤링 범위 설정
start_c_idx = 504   # 시작 C_Idx
end_c_idx = 1400   # 종료 C_Idx (1400번까지 크롤링)

# User-Agent 리스트
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
]

# 세션 생성
session = requests.Session()

# CAPTCHA 탐지 함수
def check_captcha(html):
    return "보안문자" in html or "captcha" in html.lower()  # CAPTCHA 페이지 키워드

# 크롤링 실행
for c_idx in range(start_c_idx, end_c_idx + 1):
    print(f"\n크롤링 시작: C_Idx={c_idx}")
    
    url = f"https://www.jobkorea.co.kr/starter/review/view?C_Idx={c_idx}&Ctgr_Code=5&FavorCo_Stat=0&G_ID=0"
    print(f"크롤링 중: {url}")
    
    # 랜덤 User-Agent 설정
    headers = {"User-Agent": random.choice(user_agents)}
    
    try:
        # 페이지 요청 (세션 사용)
        response = session.get(url, headers=headers)
        
        # CAPTCHA 확인
        if check_captcha(response.text):
            print("CAPTCHA 발생! 요청 중단 및 처리 대기.")
            print("CAPTCHA를 해결하고 엔터를 눌러주세요.")
            input(">> ")  # 사용자가 CAPTCHA를 해결한 후 엔터를 누름
            continue  # 다음 요청 시도
        
        if response.status_code != 200:
            print(f"페이지 요청 실패: {url}, 상태 코드: {response.status_code}")
            break
        
        # HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")
        company_tag = soup.select_one("h2.hd a[title='기업 홈 이동']")
        company_name = company_tag.get_text(strip=True) if company_tag else "회사명 없음"
        print(f"회사명: {company_name}")
        
        # 데이터 추출
        questions = soup.select("ul.qusList li div.items")
        if not questions:
            print("데이터 없음. 스킵.")
        
        for question in questions:
            title = question.select_one("span.tit").get_text(strip=True)
            content = question.select_one("span.tx").get_text(strip=True)
            print(f"{company_name} , {title} , {content}")
        
        # 요청 간 랜덤 대기
        time.sleep(random.uniform(5, 15))
        
    except Exception as e:
        print(f"오류 발생: {e}")
        continue



크롤링 시작: C_Idx=504
크롤링 중: https://www.jobkorea.co.kr/starter/review/view?C_Idx=504&Ctgr_Code=5&FavorCo_Stat=0&G_ID=0
회사명: LX MMA
LX MMA , 임원면접 - 해외영업 , 자기 소개, 한미FTA에 대하여 말씀하여 주십시요. 중국어는 가능하신가요?

크롤링 시작: C_Idx=505
크롤링 중: https://www.jobkorea.co.kr/starter/review/view?C_Idx=505&Ctgr_Code=5&FavorCo_Stat=0&G_ID=0
회사명: 회사명 없음
데이터 없음. 스킵.

크롤링 시작: C_Idx=506
크롤링 중: https://www.jobkorea.co.kr/starter/review/view?C_Idx=506&Ctgr_Code=5&FavorCo_Stat=0&G_ID=0
회사명: 버슘머트리얼즈코리아㈜
버슘머트리얼즈코리아㈜ , 일반면접 - 품질관리자 , 기숙사가 없는데 합격한다면 어쩔 것인가?
버슘머트리얼즈코리아㈜ , 일반면접 - 품질관리자 , 이직 이유는 무엇인지?
버슘머트리얼즈코리아㈜ , 일반면접 - 품질관리자 , 리더십을 발휘한 경험이 있는지?
버슘머트리얼즈코리아㈜ , 토론면접 , 업무를 하는데 동기부여가 되는것들은 뭐가 있을까요???

크롤링 시작: C_Idx=507
크롤링 중: https://www.jobkorea.co.kr/starter/review/view?C_Idx=507&Ctgr_Code=5&FavorCo_Stat=0&G_ID=0
회사명: 회사명 없음
데이터 없음. 스킵.

크롤링 시작: C_Idx=508
크롤링 중: https://www.jobkorea.co.kr/starter/review/view?C_Idx=508&Ctgr_Code=5&FavorCo_Stat=0&G_ID=0
회사명: 회사명 없음
데이터 없음. 스킵.

크롤링 시작: C_Idx=509
크롤링 중: https://www.jobkorea.co.kr/sta

In [ ]:
###### 위의 코드를 발표용 코드리뷰를 추가한 버전#########
###코드 전략 ###
# 크롤링 차단 대처: CAPTCHA 발생 시 사용자 입력 대기 및 처리.
# 효율성: 세션 및 User-Agent를 활용한 서버와의 연결 유지.
# 확장성: 크롤링 범위와 URL 생성 로직을 쉽게 변경 가능.
# 안정성: 예외 처리를 통해 크롤링 중단을 방지.

import requests
from bs4 import BeautifulSoup
import random
import time

# 크롤링 범위 설정
# C_Idx는 각 기업 리뷰 페이지를 식별하는 고유 ID
# start_c_idx부터 end_c_idx까지의 범위를 지정하여 데이터를 크롤링
start_c_idx = 1   # 시작 C_Idx
end_c_idx = 1400  # 종료 C_Idx (1400번까지 크롤링)

# User-Agent 리스트
# 요청 시 User-Agent 헤더를 설정하여 브라우저에서 보낸 요청으로 위장
# 다양한 User-Agent를 사용하여 서버의 크롤링 감지를 우회
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
]

# 세션 생성
# 세션 객체를 사용하여 요청 간 쿠키와 세션 정보를 공유
# 이를 통해 서버와의 연결을 유지하고, 로그인 또는 CAPTCHA 상태를 유지
session = requests.Session()

# CAPTCHA 탐지 함수
# HTML 내용에서 "보안문자" 또는 "captcha" 키워드가 포함되어 있는지 확인하여 CAPTCHA 페이지 여부 판단
def check_captcha(html):
    return "보안문자" in html or "captcha" in html.lower()  # CAPTCHA 페이지 키워드

# 크롤링 실행
# 지정된 C_Idx 범위를 순회하며 데이터를 크롤링
for c_idx in range(start_c_idx, end_c_idx + 1):
    print(f"\n크롤링 시작: C_Idx={c_idx}")
    
    # 크롤링할 URL 생성
    # 각 C_Idx에 해당하는 기업 리뷰 페이지의 URL을 생성
    url = f"https://www.jobkorea.co.kr/starter/review/view?C_Idx={c_idx}&Ctgr_Code=5&FavorCo_Stat=0&G_ID=0"
    print(f"크롤링 중: {url}")
    
    # 랜덤 User-Agent 설정
    # 요청 시 User-Agent를 무작위로 선택하여 크롤링 패턴을 다변화
    headers = {"User-Agent": random.choice(user_agents)}
    
    try:
        # 페이지 요청 (세션 사용)
        # 세션 객체를 통해 페이지를 요청하고 응답을 받음
        response = session.get(url, headers=headers)
        
        # CAPTCHA 확인
        # HTML 응답에 CAPTCHA가 포함된 경우 처리 대기 상태로 전환
        if check_captcha(response.text):
            print("CAPTCHA 발생! 요청 중단 및 처리 대기.")
            print("CAPTCHA를 해결하고 엔터를 눌러주세요.")
            input(">> ")  # 사용자가 CAPTCHA를 해결한 후 엔터를 누름
            continue  # 다음 요청 시도
        
        # HTTP 상태 코드 확인
        # 요청이 실패한 경우 로그를 출력하고 반복을 종료
        if response.status_code != 200:
            print(f"페이지 요청 실패: {url}, 상태 코드: {response.status_code}")
            break
        
        # HTML 파싱
        # BeautifulSoup을 사용하여 HTML 응답을 파싱하고 데이터를 추출
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 회사명 추출
        # 회사명이 포함된 태그를 선택하여 텍스트를 가져옴
        company_tag = soup.select_one("h2.hd a[title='기업 홈 이동']")
        company_name = company_tag.get_text(strip=True) if company_tag else "회사명 없음"
        print(f"회사명: {company_name}")
        
        # 면접 질문 데이터 추출
        # ul.qusList 내부의 면접 질문 데이터를 가져옴
        questions = soup.select("ul.qusList li div.items")
        if not questions:
            print("데이터 없음. 스킵.")  # 데이터가 없으면 스킵
        
        # 추출한 데이터 출력
        for question in questions:
            # 질문 제목과 내용을 추출하여 출력함
            title = question.select_one("span.tit").get_text(strip=True)
            content = question.select_one("span.tx").get_text(strip=True)
            print(f"{company_name} , {title} , {content}")
        
        # 요청 간 랜덤 대기
        # 일정 시간 대기하여 서버의 과도한 요청 감지 차단을 방지
        time.sleep(random.uniform(5, 15))
    
    except Exception as e:
        # 예외 처리
        # 크롤링 중 발생한 오류를 출력하고 다음 C_Idx로 넘어감감
        print(f"오류 발생: {e}")
        continue



In [6]:
# 전처리
import pandas as pd

# 원본 파일 경로
file_path = "C:/Users/RMARKET/Downloads/00000_Interview_AI/jobkorea_scrap.xlsx"  

# 저장할 엑셀 파일 경로
output_file_path = 'C:/Users/RMARKET/Downloads/00000_Interview_AI/processed_jobkorea_data.xlsx'  

# Excel 파일 로드
data = pd.read_excel(file_path)

# 유의미한 정보가 포함된 행만 필터링
filtered_data = data[data['Unnamed: 0'].str.contains(" , ", na=False)].reset_index(drop=True)

# '회사명', '면접 유형', '질문' 분리
processed_data = filtered_data['Unnamed: 0'].str.extract(
    r"(?P<회사명>.*?) , (?P<면접유형>.*?) , (?P<질문>.*)"
)

# '면접유형'을 '면접유형'과 '직군'으로 분리
processed_data[['면접유형', '직군']] = processed_data['면접유형'].str.split(" - ", expand=True)

# '질문'을 '질문유형'과 '면접질문'으로 분리
processed_data[['질문유형', '면접질문']] = processed_data['질문'].str.extract(r"(\[.*?\])\s*(.*)")

# 필요한 열만 정리
processed_data = processed_data[['회사명', '면접유형', '직군', '질문유형', '면접질문']]

# 엑셀 파일로 저장
processed_data.to_excel(output_file_path, index=False)

# 저장 완료 메시지
print(f"전처리된 데이터가 '{output_file_path}'에 저장되었습니다.")


전처리된 데이터가 'C:/Users/RMARKET/Downloads/00000_Interview_AI/processed_jobkorea_data.xlsx'에 저장되었습니다.


In [7]:
import pandas as pd

# 원본 데이터 로드
file_path = "C:/Users/RMARKET/Downloads/00000_Interview_AI/jobkorea_scrap.xlsx"  
data = pd.read_excel(file_path)

# 유의미한 정보가 포함된 행만 필터링
filtered_data = data[data['Unnamed: 0'].str.contains(" , ", na=False)].reset_index(drop=True)

# '회사명', '면접유형', '질문' 분리
processed_data = filtered_data['Unnamed: 0'].str.extract(
    r"(?P<회사명>.*?) , (?P<면접유형>.*?) , (?P<질문>.*)"
)

# '면접유형'을 '면접유형'과 '직군'으로 분리
processed_data[['면접유형', '직군']] = processed_data['면접유형'].str.split(" - ", expand=True)

# '질문'을 '질문유형'과 '면접질문'으로 분리
processed_data[['질문유형', '면접질문']] = processed_data['질문'].str.extract(r"(\[.*?\])?\s*(.*)")

# 결측값 확인 및 처리
processed_data.fillna(value={"질문유형": "미지정", "면접질문": "질문 없음"}, inplace=True)

# 필요한 열만 정리
processed_data = processed_data[['회사명', '면접유형', '직군', '질문유형', '면접질문']]

# 결과 저장
output_file_path = 'C:/Users/RMARKET/Downloads/00000_Interview_AI/processed_jobkorea_data.xlsx'  
processed_data.to_excel(output_file_path, index=False)

print(f"개선된 데이터가 '{output_file_path}'에 저장되었습니다.")


개선된 데이터가 'C:/Users/RMARKET/Downloads/00000_Interview_AI/processed_jobkorea_data.xlsx'에 저장되었습니다.


In [12]:
# 중복값, 결측값 확인

import pandas as pd

# 처리된 데이터 파일 로드
processed_file_path = 'C:/Users/RMARKET/Downloads/00000_Interview_AI/processed_jobkorea_data.xlsx'
processed_data = pd.read_excel(processed_file_path)

# 중복 데이터 확인
# 데이터프레임에서 중복된 행을 필터링합니다.
duplicates = processed_data[processed_data.duplicated()]

# 결측값 확인
# 각 열에 포함된 결측값(누락된 데이터)의 개수를 계산합니다.
missing_values = processed_data.isnull().sum()

# 중복 데이터 결과를 CSV 파일로 저장
duplicates.to_excel('C:/Users/RMARKET/Downloads/00000_Interview_AI/duplicates_in_processed_data.xlsx', index=False)
print("중복 데이터가 'duplicates_in_processed_data.xlsx' 파일에 저장되었습니다.")

# 결측값 정보 출력
print("결측값 정보:")
print(missing_values)

# 결측값 정보를 CSV 파일로 저장
missing_values.to_csv('C:/Users/RMARKET/Downloads/00000_Interview_AI/missing_values_info.csv', index=True)
print("결측값 정보가 'missing_values_info.csv' 파일에 저장되었습니다.")



중복 데이터가 'duplicates_in_processed_data.xlsx' 파일에 저장되었습니다.
결측값 정보:
회사명        0
면접유형       0
직군      1070
질문유형       0
면접질문       1
dtype: int64
결측값 정보가 'missing_values_info.csv' 파일에 저장되었습니다.


In [11]:
import pandas as pd

# 처리된 데이터 파일 로드
processed_file_path = 'C:/Users/RMARKET/Downloads/00000_Interview_AI/processed_jobkorea_data.xlsx'
processed_data = pd.read_excel(processed_file_path)

# 중복 데이터 확인
# 데이터프레임에서 중복된 행을 필터링합니다.
duplicates = processed_data[processed_data.duplicated()]

# 결측값 확인
# 각 열에 포함된 결측값(누락된 데이터)의 개수를 계산합니다.
missing_values = processed_data.isnull().sum()

# 중복 데이터 출력
print("\n[중복 데이터]")
if not duplicates.empty:
    print(duplicates)
else:
    print("중복된 데이터가 없습니다.")

# 결측값 정보 출력
print("\n[결측값 정보]")
print(missing_values)



[중복 데이터]
                 회사명  면접유형         직군 질문유형  \
567              ㈜KT  일반면접     화학엔지니어  미지정   
568              ㈜KT  일반면접     화학엔지니어  미지정   
569              ㈜KT  일반면접     화학엔지니어  미지정   
624         삼성중공업(주)  임원면접      엔지니어직  미지정   
1113             ㈜효성  일반면접      재무담당자  미지정   
1114             ㈜효성  일반면접      재무담당자  미지정   
1115             ㈜효성  일반면접      재무담당자  미지정   
1470       한국남부발전(주)  일반면접       환경기사  미지정   
1834       한국남동발전(주)  인성면접         사무  미지정   
1836       한국남동발전(주)  인성면접         사무  미지정   
1838       한국남동발전(주)  인성면접         사무  미지정   
2264         (주)부산은행  인성면접         행원  미지정   
2274          한화솔루션㈜  일반면접     화학엔지니어  미지정   
2275          한화솔루션㈜  일반면접     화학엔지니어  미지정   
2276          한화솔루션㈜  일반면접     화학엔지니어  미지정   
2332       동양생명보험(주)  일반면접      사무담당자  미지정   
2333       동양생명보험(주)  일반면접      사무담당자  미지정   
2334       동양생명보험(주)  일반면접      사무담당자  미지정   
6730         한솔섬유(주)  인성면접      유통-무역  미지정   
7054        신한캐피탈(주)  임원면접         영업  미지정   
7057        신한캐피탈(주)  임원

In [13]:
import pandas as pd

# 처리된 데이터 파일 로드
processed_file_path = 'C:/Users/RMARKET/Downloads/00000_Interview_AI/processed_jobkorea_data.xlsx'
processed_data = pd.read_excel(processed_file_path)

# 중복 데이터 확인
duplicates = processed_data[processed_data.duplicated()]

# 결측값 확인
missing_values = processed_data.isnull().sum()

# 결측값이 포함된 행 확인
rows_with_missing_values = processed_data[processed_data.isnull().any(axis=1)]

# 중복 데이터 출력
print("\n[중복 데이터]")
if not duplicates.empty:
    print(duplicates)
else:
    print("중복된 데이터가 없습니다.")

# 결측값 정보 출력
print("\n[결측값 정보]")
print(missing_values)

# 결측값이 포함된 행 출력
print("\n[결측값이 포함된 행]")
if not rows_with_missing_values.empty:
    print(rows_with_missing_values)
else:
    print("결측값이 포함된 행이 없습니다.")



[중복 데이터]
                 회사명  면접유형         직군 질문유형  \
567              ㈜KT  일반면접     화학엔지니어  미지정   
568              ㈜KT  일반면접     화학엔지니어  미지정   
569              ㈜KT  일반면접     화학엔지니어  미지정   
624         삼성중공업(주)  임원면접      엔지니어직  미지정   
1113             ㈜효성  일반면접      재무담당자  미지정   
1114             ㈜효성  일반면접      재무담당자  미지정   
1115             ㈜효성  일반면접      재무담당자  미지정   
1470       한국남부발전(주)  일반면접       환경기사  미지정   
1834       한국남동발전(주)  인성면접         사무  미지정   
1836       한국남동발전(주)  인성면접         사무  미지정   
1838       한국남동발전(주)  인성면접         사무  미지정   
2264         (주)부산은행  인성면접         행원  미지정   
2274          한화솔루션㈜  일반면접     화학엔지니어  미지정   
2275          한화솔루션㈜  일반면접     화학엔지니어  미지정   
2276          한화솔루션㈜  일반면접     화학엔지니어  미지정   
2332       동양생명보험(주)  일반면접      사무담당자  미지정   
2333       동양생명보험(주)  일반면접      사무담당자  미지정   
2334       동양생명보험(주)  일반면접      사무담당자  미지정   
6730         한솔섬유(주)  인성면접      유통-무역  미지정   
7054        신한캐피탈(주)  임원면접         영업  미지정   
7057        신한캐피탈(주)  임원

In [14]:
# processed data를 결측값이 있는 행을 제거한 후 저장    
import os
import pandas as pd

# 처리된 데이터 파일 로드
processed_file_path = 'C:/Users/RMARKET/Downloads/00000_Interview_AI/processed_jobkorea_data.xlsx'
processed_data = pd.read_excel(processed_file_path)

# 결측값이 포함된 행 제거
processed_data_no_missing = processed_data.dropna()

# 저장 경로 설정
output_dir = 'C:/Users/RMARKET/Downloads/00000_Interview_AI'
output_file_path = os.path.join(output_dir, 'processed_jobkorea_data_no_missing.xlsx')

# 디렉토리 생성 (존재하지 않는 경우)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 파일 저장
processed_data_no_missing.to_excel(output_file_path, index=False)

# 저장 완료 메시지
print(f"결측값이 제거된 데이터가 '{output_file_path}'에 저장되었습니다.")

결측값이 제거된 데이터가 'C:/Users/RMARKET/Downloads/00000_Interview_AI\processed_jobkorea_data_no_missing.xlsx'에 저장되었습니다.
